In [1]:
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
import os
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

In [2]:
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

#### Defining the model

In [3]:
llm = ChatGroq(
            temperature=1, 
            groq_api_key = os.environ["GROQ_API_KEY"], 
            model_name="llama-3.1-8b-instant",
            max_tokens=600,
            top_p=0.90,
        #     frequency_penalty=1,
        #     presence_penalty=1,
    )

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


#### Embbeding and using vector database

In [4]:
embedding = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1",model_kwargs={"device": "cuda"})
chroma_db = Chroma(
    persist_directory="./chroma_capstone_db_new_small",
    embedding_function=embedding,
    collection_name="Tomato"  # Specify which collection to load czx
)

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Chromadb as Retriever using mmr

In [5]:
question = "give me the cure for tomato plant in Kashmir in the summer months"
chroma_retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 6, "fetch_k":12})

chroma_retriever.get_relevant_documents(question)

C:\Users\HP\AppData\Local\Temp\ipykernel_22112\3181224039.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_retriever.get_relevant_documents(question)


[Document(id='7a4e23f8-cc6d-4858-bafa-104203d9dde8', metadata={'StateName': 'ASSAM', 'disease_name': 'Late blight'}, page_content='DistrictName:DHUBRI\nStateName:ASSAM\nSeason_English:Monsoon Crop\nMonth:November\nDisease:Late blight\nQueryText:ASKING FOR ANY PLANT PROTECTION MEASURES TO BE ADOPTED AGAINST LATE BLIGHT DISEASE IN TOMATO\nKccAns:INDOFIL M-45 SOLUTION  25 GRAMS PER 1 LITRE OF WATER HAS BEEN SUGGESTED TO SPRAY ON THE CROP AT THE FOLLOWING STAGES: 1 FIRST AT 30 DAYS AFTER TRANSPLANTING 2 SECOND AT 60 DAYS AFTER TRANSPLANTING AND 3 THIRD AT 90 DAYS AFTER TRANSPLANTING AS THE PLANT PROTECTION MEASURES TO BE ADOPTED AGAINST LATE BLIGHT DISEASE IN TOMATO'),
 Document(id='2740b954-c961-4c6b-96bc-a5e30e496354', metadata={'StateName': 'JAMMU AND KASHMIR', 'disease_name': 'Aphids'}, page_content='DistrictName:BADGAM\nStateName:JAMMU AND KASHMIR\nSeason_English:Monsoon Crop\nMonth:April\nDisease:Aphids\nQueryText:HOW TO CONTROL APHIDS IN THGE SEEDLINGS OF TOMATO \nKccAns:SPRAY MELAT

#### Prompting

In [6]:
prompt_template = """
You are an agricultural assistant specialized in answering questions about plant diseases.  
Your task is to provide answers strictly based on the provided context when possible.  

Each document contains the following fields:  
- DistrictName  
- StateName  
- Season_English  
- Month  
- Disease  
- QueryText  
- KccAns (this is the official response section from source documents)

Guidelines for answering:
1. If a relevant answer is available in KccAns, use that with minimal changes.
2. Use DistrictName, StateName, Season_English, Month, and Disease only to help interpret the question and select the correct KccAns, but **do not include these details in the final answer unless the question explicitly asks for them**.  
3. If the answer is not available in the context, then rely on your own agricultural knowledge to provide the best possible answer.  
4. Do not invent or assume information when KccAns is present; only fall back to your own knowledge when the context has no suitable answer.  

CONTEXT:
{context}

QUESTION:
{question}

OUTPUT:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

#### Retriever

In [7]:
h_retrieval_QA1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [8]:
question = "What are some of the common prescriptions for tomato plants in Karnataka"
print(h_retrieval_QA1.invoke({"query": question})["result"])

Based on the provided context, the following are some common prescriptions for tomato plants in Karnataka:

1. For Early and Late Blight of Tomato:
   - Drench 0.5g K-cyclene or 0.5g Streptocyclene with 30g Copper oxychloride in a liter of water at the base of the affected plants.
   - Spray 20g Dimethomorph (Polyram) in a liter of water 5 weeks after transplanting.
   - Repeat the spray on the 7th, 9th, and 11th weeks after transplanting.
   - 200-250 liters of spray solution required per acre.

2. For Late Blight in Tomato:
   - Metalaxyl-M 4 + Mancozeb 64 WG - 2g/liter OR Chlorothalonil 250g/100Ltr of water.

3. For Leaf Miner in Tomato:
   - Spray 20ml Triazophos 40 EC/liter of water.

4. For Fruit Borer in Tomato:
   - Spray Chlorantraniliprole 5ml/10Ltr of water.

Note that these prescriptions are based on the provided context and might not be an exhaustive list of all possible treatments for tomato plants in Karnataka. It is always best to consult with a qualified agricultural e

In [9]:
Q2 = "I have a Fruit borer in my Tomato, what prescriptions would you recommend in Madhya Pradesh"
print(h_retrieval_QA1.invoke({"query": Q2})["result"])

Since you have a fruit borer in your tomato crop and you are in Madhya Pradesh, I will rely on the provided context to find the most suitable answer. 

Upon reviewing the given contexts, I found that there are several recommendations for controlling fruit borer in tomato crops in Madhya Pradesh. 

Some of the recommended prescriptions are:

- Use recommended dose of chloropyriphos 20EC 800 ML to 1 liter mixed in 8-10 kg of sand and broad cast in 1 acre of field and Carbosulfan 25 EC 25-30 ml per pump.
- Spray Indoxacarb 145 SC 10-12 ML per sprayer or Flubendiamide 480 SC 4-5 ML per sprayer.
- Use Indoxacarb 145 SC 10 ml per 15 liters of water.
- Spray the solution of Quinalphose 25EC 1 ml per liter, Indoxacarb 145 SC 13-15 ml per pump, Trizophos 30ml per pump, chlorpyriphos 1ml per liter, or Emamectin benzoate 5 SG.
- Spray Trizofose 30 ml per pump.

These prescriptions are based on the given contexts and are suitable for controlling fruit borer in tomato crops in Madhya Pradesh. Howev